# SQL:

Yuanfeng Li 
2021/07/22

### Review support materials:
- [sql 复习汇总](https://zhuanlan.zhihu.com/p/43810990)
- [Leetcode Database 全部练习题及题解](https://zhuanlan.zhihu.com/p/265247944)

In [ ]:
# 175, combine two tables:
# My Answer is wrong, because I did not understand 'regardless if there is an address for each of those people'
# Correct Answer: 

# Write your MySQL query statement below
SELECT P.FirstName, P.LastName, A.City, A.State
FROM Person as P LEFT JOIN Address as A
ON P.PersonId = A.PersonId;

In [ ]:
# 176:

# Leetcode answer:
SELECT
 (SELECT DISTINCT Salary
  FROM Employee
  ORDER BY Salary DESC
  LIMIT 1 OFFSET 1) 
AS SecondHighestSalary;

# my version 1 year ago
# Write your MySQL query statement below
SELECT IFNULL(
    (SELECT DISTINCT Salary 
    FROM Employee 
    ORDER BY Salary DESC
    LIMIT 1 OFFSET 1), NULL
) AS SecondHighestSalary;


# my current version is not right, 
# I forget that limit 1 offset 1
SELECT DISTINCT Salary AS SecondHighestSalary
  FROM Employee
  ORDER BY Salary DESC
  LIMIT 1 OFFSET 1
# this one will not return null, it will return an empty 


In [ ]:
# 177
# tutorial url: https://www.mysqltutorial.org/mysql-limit.aspx

CREATE FUNCTION getNthHighestSalary(N INT) RETURNS INT
BEGIN
    DECLARE temp INTEGER;
    SET temp = N-1;
  RETURN (
      # Write your MySQL query statement below.
      SELECT IFNULL(
          (SELECT DISTINCT Salary 
           FROM Employee 
           ORDER BY Salary DESC 
           LIMIT temp,1), 
          NULL)
  );
END

In [ ]:
# 1241. Number of Comments per post

